##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# 保存和恢復模型 

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://tensorflow.org/tutorials/keras/save_and_load"><img src="https://tensorflow.org/images/tf_logo_32px.png" />在 tensorflow.org 上查看</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_load.ipynb"><img src="https://tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 運行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_load.ipynb"><img src="https://tensorflow.org/images/GitHub-Mark-32px.png" />在 Github 上查看源代碼</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/save_and_load.ipynb"><img src="https://tensorflow.org/images/download_logo_32px.png" />下載此 notebook</a>
  </td>
</table>

Note: 我們的 TensorFlow 社區翻譯了這些文檔。因為社區翻譯是盡力而為， 所以無法保證它們是最準確的，並且反映了最新的
[官方英文文檔](https://www.tensorflow.org/?hl=en)。如果您有改進此翻譯的建議， 請提交 pull request 到
[tensorflow/docs](https://github.com/tensorflow/docs) GitHub 倉庫。要志願地撰寫或者審核譯文，請加入
[docs@tensorflow.org Google Group](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs)。

模型可以在訓練期間和訓練完成後進行保存。這意味著模型可以從任意中斷中恢復，並避免耗費比較長的時間在訓練上。保存也意味著您可以共享您的模型，而其他人可以通過您的模型來重新創建工作。在發布研究模型和技術時，大多數機器學習從業者分享：

* 用於創建模型的代碼
* 模型訓練的權重 (weight) 和參數 (parameters) 。

共享數據有助於其他人了解模型的工作原理，並使用新數據自行嘗試。

注意：小心不受信任的代碼——Tensorflow 模型是代碼。有關詳細信息，請參閱 [安全使用Tensorflow](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md)。

### 選項

保存 Tensorflow 的模型有許多方法——具體取決於您使用的 API。本指南使用 [tf.keras](https://tensorflow.org/guide/keras)， 一個高級 API 用於在 Tensorflow 中構建和訓練模型。有關其他方法的實現，請參閱 TensorFlow [保存和恢復](https://tensorflow.org/guide/saved_model)指南或[保存到 eager](https://tensorflow.org/guide/eager#object-based_saving)。

## 配置

### 安裝並導入

安裝並導入Tensorflow和依賴項：

In [0]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass

!pip install pyyaml h5py  # 需要以 HDF5 格式保存模型

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

### 獲取示例數據集

要演示如何保存和加載權重，您將使用 [MNIST 數據集](http://yann.lecun.com/exdb/mnist/). 要加快運行速度，請使用前1000個示例：

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### 定義模型

首先構建一個簡單的序列（sequential）模型：

In [0]:
# 定義一個簡單的序列模型
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

# 創建一個基本的模型實例
model = create_model()

# 顯示模型的結構
model.summary()

## 在訓練期間保存模型（以 checkpoints 形式保存）

您可以使用訓練好的模型而無需從頭開始重新訓練，或在您打斷的地方開始訓練，以防止訓練過程沒有保存。 `tf.keras.callbacks.ModelCheckpoint` 允許在訓練的*過程中*和*結束時*回調保存的模型。

### Checkpoint 回調用法

創建一個只在訓練期間保存權重的 `tf.keras.callbacks.ModelCheckpoint` 回調：

In [0]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 創建一個保存模型權重的回調
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# 使用新的回調訓練模型
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 通过回调训练

# 這可能會生成與保存優化程序狀態相關的警告。
# 這些警告（以及整個筆記本中的類似警告）是防止過時使用，可以忽略。

這將創建一個 TensorFlow checkpoint 文件集合，這些文件在每個 epoch 結束時更新：

In [0]:
!ls {checkpoint_dir}

創建一個新的未經訓練的模型。僅恢復模型的權重時，必須具有與原始模型具有相同網絡結構的模型。由於模型具有相同的結構，您可以共享權重，儘管它是模型的不同*實例*。
現在重建一個新的未經訓練的模型，並在測試集上進行評估。未經訓練的模型將在機會水平（chance levels）上執行（準確度約為10％）：

In [0]:
# 創建一個基本模型實例
model = create_model()

# 評估模型
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

然後從 checkpoint 加載權重並重新評估：

In [0]:
# 加載權重
model.load_weights(checkpoint_path)

# 重新評估模型
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

### checkpoint 回調選項

回調提供了幾個選項，為 checkpoint 提供唯一名稱並調整 checkpoint 頻率。

訓練一個新模型，每五個 epochs 保存一次唯一命名的 checkpoint：

In [0]:
# 在文件名中包含 epoch (使用 `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 創建一個回調，每 5 個 epochs 保存模型的權重
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# 創建一個新的模型實例
model = create_model()

# 使用 `checkpoint_path` 格式保存權重
model.save_weights(checkpoint_path.format(epoch=0))

# 使用新的回調*訓練*模型
model.fit(train_images, 
              train_labels,
              epochs=50, 
              callbacks=[cp_callback],
              validation_data=(test_images,test_labels),
              verbose=0)

現在查看生成的 checkpoint 並選擇最新的 checkpoint ：

In [0]:
! ls {checkpoint_dir}

In [0]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

注意: 默認的 tensorflow 格式僅保存最近的5個 checkpoint 。

如果要進行測試，請重置模型並加載最新的 checkpoint ：

In [0]:
# 創建一個新的模型實例
model = create_model()

# 加載以前保存的權重
model.load_weights(latest)

# 重新評估模型
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## 這些文件是什麼？

上述代碼將權重存儲到 [checkpoint](https://tensorflow.org/guide/saved_model#save_and_restore_variables)—— 格式化文件的集合中，這些文件僅包含二進制格式的訓練權重。 Checkpoints 包含：
* 一個或多個包含模型權重的分片。
* 索引文件，指示哪些權重存儲在哪個分片中。

如果你只在一台機器上訓練一個模型，你將有一個帶有後綴的碎片： `.data-00000-of-00001`

## 手動保存權重

您將了解如何將權重加載到模型中。使用 `Model.save_weights` 方法手動保存它們同樣簡單。默認情況下， `tf.keras` 和 `save_weights` 特別使用 TensorFlow [checkpoints](../../guide/keras/checkpoints) 格式 `.ckpt` 擴展名和 ( 保存在 [HDF5](https://js.tensorflow.org/tutorials/import-keras.html) 擴展名為 `.h5` [保存并序列化模型](../../guide/keras/save_and_serialize#weights-only_saving_in_savedmodel_format) )：

In [0]:
# 保存權重
model.save_weights('./checkpoints/my_checkpoint')

# 創建模型實例
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## 保存整個模型

模型和優化器可以保存到包含其狀態（權重和變量）和模型參數的文件中。這可以讓您導出模型，以便在不訪問原始 python 代碼的情況下使用它。而且您可以通過恢復優化器狀態的方式，從中斷的位置恢復訓練。

保存完整模型會非常有用——您可以在 TensorFlow.js ([HDF5](https://js.tensorflow.org/tutorials/import-keras.html), [Saved Model](https://js.tensorflow.org/tutorials/import-saved-model.html)) 加載他們，然後在 web 瀏覽器中訓練和運行它們，或者使用 TensorFlow Lite 將它們轉換為在移動設備上運行([HDF5](https://tensorflow.org/lite/convert/python_api#exporting_a_tfkeras_file_), [Saved Model](https://tensorflow.org/lite/convert/python_api#exporting_a_savedmodel_))

### 將模型保存為HDF5文件

Keras 可以使用 [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) 標準提供基本保存格式。出於我們的目的，可以將保存的模型視為單個二進制blob：

In [0]:
# 創建一個新的模型實例
model = create_model()

# 訓練模型
model.fit(train_images, train_labels, epochs=5)

# 將整個模型保存為HDF5文件
model.save('my_model.h5')

現在，從該文件重新創建模型：

In [0]:
# 重新創建完全相同的模型，包括其權重和優化程序
new_model = keras.models.load_model('my_model.h5')

# 顯示網絡結構
new_model.summary()

檢查其準確率（accuracy）：

In [0]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

這項技術可以保存一切:

* 權重
* 模型配置(結構)
* 優化器配置

Keras 通過檢查網絡結構來保存模型。目前，它無法保存 Tensorflow 優化器（調用自 `tf.train`）。使用這些的時候，您需要在加載後重新編譯模型，否則您將失去優化器的狀態。


### 通過 `saved_model` 保存

注意：這種保存 `tf.keras` 模型的方法是實驗性的，在將來的版本中可能有所改變。

建立一個新模型，然後訓練它：

In [0]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

創建一個 `saved_model`，並將其放在帶有 `tf.keras.experimental.export_saved_model` 的帶時間戳的目錄中：

In [0]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

列出您保存的模型：

In [0]:
!ls saved_models/

從保存的模型重新加載新的 Keras 模型：

In [0]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)

# 顯示網絡結構
new_model.summary()

使用恢復的模型運行預測：

In [0]:
model.predict(test_images).shape

In [0]:
# 必須在評估之前編譯模型。
# 如果僅部署已保存的模型，則不需要此步驟。

new_model.compile(optimizer=model.optimizer,  # 保留已加載的優化程序
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

# 評估已恢復的模型
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))